In [ ]:
# 扩展是将短文本（例如一组说明或主题列表）输入到大型语言模型中，让模型生成更长的文本（例如基于某个主题的电子邮件或论文）。
# 这种应用是一把双刃剑，好处例如将大型语言模型用作头脑风暴的伙伴；但也存在问题，例如某人可能会使用它来生成大量垃圾邮件。
# 因此，当你使用大型语言模型的这些功能时，请仅以负责任 (responsible) 和有益于人们 (helps people) 的方式使用它们。
# 在本章中，你将学会如何基于 OpenAI API 生成针对每位客户评价优化的客服电子邮件。我们还将利用模型的另一个输入参数称为温
# 度，这种参数允许您在模型响应中变化探索的程度和多样性。
# 同以上几章，你需要类似的代码来配置一个可以使用 OpenAI API 的环境

In [1]:
from openai import OpenAI
def get_completion_gpt(prompt, model="gpt-4o-mini", temperature=0): 
    messages = [{"role": "user", "content": prompt}] # 消息队列
    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # 值越低则输出文本随机性越低
    )
    return response.choices[0].message.content

In [ ]:
# 定制客户邮件
# 我们将根据客户评价和情感，针对性写自动回复邮件。因此，我们将给定客户评价和情感，使用 LLM 针对性生成响应，
# 即根据客户评价和评论情感生成定制电子邮件。
# 我们首先给出一个示例，包括一个评论及对应的情感。

In [ ]:
# 鉴于“推断”课程中的观点，和原始客户留言一样，自定义邮件
sentiment = "negative" # 情绪:消极
# 搅拌机评论
review = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [2]:
# 我们可以在推理那章学习到如何对一个评论判断其情感倾向
sentiment = "negative" # 首先要调用模型对评论推断出情感倾向
# 之后根据prompt,prompt里面包含评论和情感倾向的字符串,让模型明白它要看的是哪个评论和情感倾向
review = """
他们在11月份的季节性销售期间以约49美元的价格出售17件套装，折扣约为一半。\
但由于某些原因（可能是价格欺诈），到了12月第二周，同样的套装价格全都涨到了70美元到89美元不等。\
11件套装的价格也上涨了大约10美元左右。\
虽然外观看起来还可以，但基座上锁定刀片的部分看起来不如几年前的早期版本那么好。\
不过我打算非常温柔地使用它，例如，\
我会先在搅拌机中将像豆子、冰、米饭等硬物研磨，然后再制成所需的份量，\
切换到打蛋器制作更细的面粉，或者在制作冰沙时先使用交叉切割刀片，然后使用平面刀片制作更细/不粘的效果。\
制作冰沙时，特别提示：\
将水果和蔬菜切碎并冷冻（如果使用菠菜，则轻轻煮软菠菜，然后冷冻直到使用；\
如果制作果酱，则使用小到中号的食品处理器），这样可以避免在制作冰沙时添加太多冰块。\
大约一年后，电机发出奇怪的噪音，我打电话给客服，但保修已经过期了，所以我不得不再买一个。\
总的来说，这些产品的总体质量已经下降，因此它们依靠品牌认可和消费者忠诚度来维持销售。\
货物在两天内到达。
"""

In [ ]:
# 我们已经使用推断课程中所学方法提取了情感，这是一个关于搅拌机的客户评价，现在我们将根据情感定制回复。
# 以下述 Prompt 为例：假设你是一个客户服务 AI 助手，你的任务是为客户发送电子邮件回复，根据通过三个
# 反引号分隔的客户电子邮件，生成一封回复以感谢客户的评价。
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion_gpt(prompt)
print(response)

In [4]:
# 其实prompt还是个字符串,只不过你必须让LLM明白你的需求
prompt = f"""
你是一位客户服务的AI助手。
你的任务是给一位重要客户发送邮件回复。
根据通过```分隔的客户评价，生成回复以感谢客户的评价。提醒模型使用评价中的具体细节
用简明而专业的语气写信。
作为“AI客户代理”签署电子邮件。
客户评论：
```{review}```
评论情感：{sentiment}
"""
response = get_completion_gpt(prompt)
print(response)

主题：感谢您的反馈

尊敬的客户，

感谢您抽出时间分享您的反馈。我们非常重视您的意见，并对您在11月份季节性销售期间的购物体验表示遗憾。我们注意到您提到的价格波动，以及您对产品质量的担忧，尤其是基座上锁定刀片的部分。

我们理解您对产品质量下降的失望，并感谢您提供的使用建议，这将帮助我们改进产品设计和用户体验。我们也很高兴得知您对配送速度的满意。

请您放心，我们会将您的反馈传达给相关团队，以便进一步评估和改进我们的产品和服务。如果您有任何其他问题或需要进一步的帮助，请随时与我们联系。

再次感谢您的宝贵意见。

祝好，

AI客户代理


In [ ]:
print(prompt)

In [ ]:
# 引入温度系数
# 接下来，我们将使用语言模型的一个称为“温度” (Temperature) 的参数，它将允许我们改变模型响应的多样性。您可以将温度
# 视为模型探索或随机性的程度。
# 例如，在一个特定的短语中，“我的最爱食品”最有可能的下一个词是“比萨”，其次最有可能的是“寿司”和“塔可”。因此，在温度为
# 零时，模型将总是选择最有可能的下一个词，而在较高的温度下，它还将选择其中一个不太可能的词，在更高的温度下，它甚至可能
# 选择塔可，而这种可能性仅为五分之一。您可以想象，随着模型继续生成更多单词的最终响应，“我的最爱食品是比萨”将会与第一个
# 响应“我的最爱食品是塔可”产生差异。随着模型的继续，这两个响应也将变得越来越不同。
# 一般来说，在构建需要可预测响应的应用程序时，我建议设置温度为零。在所有课程中，我们一直设置温度为零，如果您正在尝试构
# 建一个可靠和可预测的系统，我认为您应该选择这个温度。如果您尝试以更具创意的方式使用模型，可能需要更广泛地输出不同的结果
# ，那么您可能需要使用更高的温度。
# 同一段来信，我们提醒模型使用用户来信中的详细信息，并设置温度：

In [ ]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion_gpt(prompt, temperature=0.7)
print(response)

In [5]:
prompt = f"""
你是一名客户服务的AI助手。
你的任务是给一位重要的客户发送邮件回复。
根据通过“```”分隔的客户电子邮件生成回复，以感谢客户的评价。
如果情感是积极的或中性的，感谢他们的评价。
如果情感是消极的，道歉并建议他们联系客户服务。
请确保使用评论中的具体细节。
以简明和专业的语气写信。
以“AI客户代理”的名义签署电子邮件。
客户评价：```{review}```
评论情感：{sentiment}
"""
response = get_completion_gpt(prompt, temperature=0.7)
print(response)

主题：感谢您的反馈

尊敬的客户，

感谢您抽出时间分享您的评价。我们很遗憾地得知您对产品和价格调整的经历感到失望。我们始终致力于提供高质量的产品和服务，因此您提到的质量下降和价格变化让我们非常重视。

对于您在使用产品过程中遇到的电机噪音问题，我们深表歉意。虽然保修已过期，但我们建议您直接联系客户服务团队，以便我们进一步了解您的情况并提供帮助。

感谢您对我们品牌的支持与信任，我们期待在未来为您提供更好的体验。

顺祝商祺，

AI客户代理


In [6]:
review

'\n他们在11月份的季节性销售期间以约49美元的价格出售17件套装，折扣约为一半。但由于某些原因（可能是价格欺诈），到了12月第二周，同样的套装价格全都涨到了70美元到89美元不等。11件套装的价格也上涨了大约10美元左右。虽然外观看起来还可以，但基座上锁定刀片的部分看起来不如几年前的早期版本那么好。不过我打算非常温柔地使用它，例如，我会先在搅拌机中将像豆子、冰、米饭等硬物研磨，然后再制成所需的份量，切换到打蛋器制作更细的面粉，或者在制作冰沙时先使用交叉切割刀片，然后使用平面刀片制作更细/不粘的效果。制作冰沙时，特别提示：将水果和蔬菜切碎并冷冻（如果使用菠菜，则轻轻煮软菠菜，然后冷冻直到使用；如果制作果酱，则使用小到中号的食品处理器），这样可以避免在制作冰沙时添加太多冰块。大约一年后，电机发出奇怪的噪音，我打电话给客服，但保修已经过期了，所以我不得不再买一个。总的来说，这些产品的总体质量已经下降，因此它们依靠品牌认可和消费者忠诚度来维持销售。货物在两天内到达。\n'

In [7]:
prompt = f"""
你是一名客户服务的AI助手。
你的任务是给一位重要的客户发送邮件回复。
根据通过```分隔的客户电子邮件生成回复，以感谢客户的评价。
如果情感是积极的或中性的，感谢他们的评价。
如果情感是消极的，道歉并建议他们联系客户服务。
请确保使用评论中的具体细节。
以简明和专业的语气写信。
以“AI客户代理”的名义签署电子邮件。
客户评价：```{review}```
评论情感：{sentiment}
"""
response = get_completion_gpt(prompt, temperature=0.7)
print(response)

主题：感谢您的反馈

尊敬的客户，

感谢您抽出时间分享您的反馈意见。我们非常重视客户的体验，并对您在使用我们产品过程中遇到的问题表示诚挚的歉意。

我理解您对价格变化和产品质量的失望，尤其是关于刀片基座的设计以及电机噪音的问题。我们一直在努力改进我们的产品，如果您有进一步的建议或需要帮助，请随时联系客户服务团队，我们将竭诚为您提供支持。

再次感谢您的宝贵反馈，期待能有机会为您提供更好的服务。

祝好，

AI客户代理


In [ ]:
# 在温度为零时，每次执行相同的 Prompt ，您获得的回复理应相同。而使用温度为 0.7 时，则每次都会获得不同的输出。
# 所以，您可以看到它与我们之前收到的电子邮件不同。再次执行将再次获得不同的电子邮件。
# 因此，我建议您自己尝试温度，以查看输出如何变化。总之，在更高的温度下，模型的输出更加随机。您几乎可以将其视为在
# 更高的温度下，助手更易分心，但也许更有创造力。